# MAL Basics
* This notebook is a high level introduction demonstrating that you can upload inferences from python objects, local files, and remote files.
* For complete examples see image_mal.ipynb or ner_mal.ipynb

* There are three ways to import annotations using the upload_annotations method.
    1. For images, the Model-assisted labeling workflow supports all annotation types, except Dropdown classification and classifications nested within classifications.
    2. For videos, the Model-assisted workflow only supports classifications at the frame-level.
    3. For text, the Model-assisted workflow supports Named entity recognition and text classification.

* Hints:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

In [9]:
!pip install labelbox
!pip install labelbox[data]
!pip install requests
!pip install ndjson
!pip install pathlib

In [2]:
from labelbox import Client
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend
from pathlib import Path
import ndjson
import os
import requests
import uuid
import json
from getpass import getpass

In [5]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [6]:
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [7]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

In [6]:
# Only update this if you have an on-prem deployment
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="person"),
])
project = client.create_project(name="image_mal_project")
dataset = client.create_dataset(name="image_mal_dataset")
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
project.setup(editor, ontology_builder.asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()
ontology = ontology_builder.from_project(project)
feature_schema_id = ontology.tools[0].feature_schema_id

#### Collect annotations
* Can use any of the following approaches:
1. Python Iterable of dicts
2. NDjson stored locally
3. NDjson accessible via a public/signed url

* Whether the data is a file or an iterable of dicts it must follow the structure defined here:
https://docs.labelbox.com/data-model/en/index-en#annotations

* Python objects

In [7]:
#For more details see image_mal.ipynb or ner_mal.ipynb
annotations = [{
    "uuid": str(uuid.uuid4()),
    "schemaId": feature_schema_id,
    "dataRow": {
        "id": data_row.uid,
    },
    "bbox": {
        "top": int(30),
        "left": int(30),
        "height": 200,
        "width": 200
    }
}]

* Local NDjson file

In [8]:
with open('/tmp/example.ndjson', 'w') as file:
    file.write(ndjson.dumps(annotations))

* Remote NDjson file

In [9]:
with open("/tmp/example.ndjson") as file:
    content = file.read()
ndjson_url = client.upload_data(content=content, sign=True)

### Perform the upload
* Choose one of the following to run
* Then run the cell with upload_job.wait_until_done()
* Once this completes (up to five minuntes), you can see the annotations if you go to the link that is printed out below

### MAL from a python object

In [10]:
upload_job = project.upload_annotations(name="upload_py_object_job",
                                        annotations=annotations)

### MAL from a file

In [11]:
predictions_file = Path("/tmp/example.ndjson")
upload_job = project.upload_annotations(name="upload_local_ndjson_job",
                                        annotations=predictions_file)

### MAL from a url

In [12]:
upload_job = project.upload_annotations(name="upload_remote_ndjson_job",
                                        annotations=ndjson_url)

Validation is turned on. The file will be downloaded locally and processed before uploading.


In [13]:
# Wait for the jobs to finish
upload_job.wait_until_done()
#See https://github.com/Labelbox/labelbox-python/blob/develop/labelbox/schema/enums.py
# for possible states
print("State", upload_job.state)
print(
    "Visit the following and click start labeling to see the uploaded example:",
    f"https://app.labelbox.com/projects/{project.uid}")

Visit the following and click start labeling to see the uploaded example: https://app.labelbox.com/projects/ckmuilp77c0ov0789ome3arln


If you are uploading a public URL to an NDJSON file,
check that the host of the public URL allows standard browsers to download by doing the following:
* Navigate to your URL using any browser. It should return the expected NDJSON.
* Run wget -O- --user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36  (KHTML, like Gecko) Chrome/81.0.4044.138  Safari/537.36' <url> | cat. It should return the expected NDJSON.

In [14]:
os.environ['upload_file_url'] = ndjson_url
!wget -O- --user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36  (KHTML, like Gecko) Chrome/81.0.4044.138  Safari/537.36' \
      $upload_file_url | cat

--2021-03-29 07:30:40--  https://storage.labelbox.com/ckk4q1vgapsau07324awnsjq2%2Fa6bcb5b1-28f6-2719-bba5-2af5a09508ec-1?Expires=1617103411913&KeyName=labelbox-assets-key-1&Signature=pszklnoZKifP7HTvXTG4gVnHVbs
Resolving storage.labelbox.com (storage.labelbox.com)... 34.96.97.159
Connecting to storage.labelbox.com (storage.labelbox.com)|34.96.97.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199 [text/plain]
Saving to: ‘STDOUT’

{"uuid": "fd14e9ef-6bf9-4300-bf2a-7e85f4f0b009", "schemaId": "ckmui64il0s4d0y5rfjvd8v4v", "dataRow": {"id": "ckmui63lo49gd0y62fampaew4"}, "bbox": {"top": 30, "left": 30, "height": 200, "width": 200}}-                   100%[===================>]     199  --.-KB/s    in 0s      

2021-03-29 07:30:40 (63.3 MB/s) - written to stdout [199/199]



In [15]:
# Error for each annotation that failed.
# Empty list means that there were no errors
print("Errors:", upload_job.errors)
# Status for each ndjson annotation
print("Statuses", upload_job.statuses)
# The ndjson you uploaded
print("inputs", upload_job.inputs)